In [1]:
# The basics
import numpy as np
import pandas as pd

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# Keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, Softmax, LSTM

# utils
import os

In [2]:
isear = pd.read_csv('../data/raw/isear.csv', sep='|', error_bad_lines=False, usecols=['Field1', 'SIT', 'EMOT'])

In [3]:
number_of_classes = len(isear.EMOT.unique())

In [4]:
maxlen = 1000
max_words = 10000

In [5]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(isear['SIT'])
sequences = tokenizer.texts_to_sequences(isear['SIT'])

In [6]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 9063 unique tokens.


In [7]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post')

In [8]:
x_train, x_test, y_train, y_test = train_test_split(data, isear['EMOT'])

In [9]:
glove_dir = '../data/external'
embeddings_index = {}

f = open(os.path.join(glove_dir, 'glove.6B.50d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [10]:
embeddings_index['king'] - embeddings_index['man'] + embeddings_index['woman'] - embeddings_index['queen']

array([ 0.03882596, -0.91902995,  0.25977004,  0.04227898,  0.13896999,
        0.20638007,  0.02683   , -0.03402001, -0.09974198,  0.08821005,
        0.007544  , -0.36111003,  0.320001  , -0.6568099 ,  0.63689005,
        0.33902   , -0.7714    , -0.59767103,  0.711054  ,  0.44857004,
       -0.08957994, -0.01644999, -0.08847399, -0.17911002,  0.43223003,
       -0.28219986, -0.09410012, -0.9570599 ,  0.01775998,  0.32842597,
        0.11489999, -0.36365002, -0.52906007,  0.11805284, -0.10499001,
       -0.219787  , -0.33020002, -0.137476  ,  0.10705006,  0.92508006,
       -0.223916  ,  0.71121997, -0.29156998,  0.24670503, -0.05374995,
        0.2274299 , -0.09986001,  0.33875012, -0.69724536,  0.10499996],
      dtype=float32)

In [11]:
embedding_dim = 50 # if chaning this, update the file name above 

embedding_matrix = np.zeros((max_words, embedding_dim))

for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## Model creation time

In [19]:
model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
#model.add(Flatten())
model.add(LSTM(128))
model.add(Dense(number_of_classes + 1,  activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 1000, 50)          500000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               91648     
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1032      
Total params: 592,680
Trainable params: 592,680
Non-trainable params: 0
_________________________________________________________________


In [24]:
model.fit(x_train, to_categorical(y_train),
          epochs=5,
          batch_size=32,
          validation_data=(x_test, to_categorical(y_test)))

Train on 5749 samples, validate on 1917 samples
Epoch 1/5
5749/5749 [==============================] - 122s 21ms/sample - loss: 1.9494 - accuracy: 0.1452 - val_loss: 1.9483 - val_accuracy: 0.1497
Epoch 2/5
5749/5749 [==============================] - 117s 20ms/sample - loss: 1.9498 - accuracy: 0.1419 - val_loss: 1.9503 - val_accuracy: 0.1346
Epoch 3/5
5749/5749 [==============================] - 114s 20ms/sample - loss: 1.9484 - accuracy: 0.1494 - val_loss: 1.9480 - val_accuracy: 0.1466
Epoch 4/5
5749/5749 [==============================] - 117s 20ms/sample - loss: 1.9501 - accuracy: 0.1409 - val_loss: 1.9466 - val_accuracy: 0.1450
Epoch 5/5
5749/5749 [==============================] - 119s 21ms/sample - loss: 1.9492 - accuracy: 0.1259 - val_loss: 1.9472 - val_accuracy: 0.1382


In [25]:
y_pred = model.predict_classes(x_test)
y_pred

array([5, 5, 5, ..., 5, 5, 5])

In [26]:
confusion_matrix(y_test, y_pred)

array([[  0,   0,   0,   0, 282,   0,   0],
       [  0,   0,   0,   0, 258,   0,   0],
       [  0,   0,   0,   0, 281,   0,   0],
       [  0,   0,   0,   0, 287,   0,   0],
       [  0,   0,   0,   0, 265,   0,   0],
       [  0,   0,   0,   0, 278,   0,   0],
       [  0,   0,   0,   0, 266,   0,   0]])